# Drive characteristics using `strym`

In this notebook, we will generate characteristics of a particular drive. Please ensure strym is installed correctly

## Installing `strym`
First, we are required to install `strym`. For that, we will just use github devtools functionality.
Even if you have installed `strym` previously, you should reinstall for latest version of the package.

In [67]:
# Installing strym using pip from git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym

  Cloning https://github.com/jmscslgroup/strym to /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-req-build-beww1_nv
  Running command git clone -q https://github.com/jmscslgroup/strym /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-req-build-beww1_nv
  Created wheel for strym: filename=strym-0.1-cp37-none-any.whl size=19497 sha256=5d2307695f1b9342691c1e877220c65b4fb504a81866091fd27fed40277af038
  Stored in directory: /private/var/folders/qb/xzd2v2fn741b459s8jvfjhk4000zl1/T/pip-ephem-wheel-cache-en2ca1es/wheels/16/8c/7f/7ab5f7b26c83fdbe3972de24a6aeb113cf4a35c9125d065c9d
Successfully built strym
  Found existing installation: strym 0.1
    Uninstalling strym-0.1:
      Successfully uninstalled strym-0.1


## Importing packages
Import required packages

In [97]:
from strym import strymread
import strym
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate

## Specify Data Location

In [301]:
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
# import glob
# csvlist = glob.glob(datafolder+"*.csv")
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-05/"
# # print(csvlist)
# for f in glob.glob(datafolder+"*.csv"):
#     csvlist.append(f)
# print(csvlist)

In [305]:
parentfolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/"
folderlist = glob.glob(parentfolder+"*")
print(folderlist)
csvlist = []
for datafolder in folderlist:
#     datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
    import glob
    csvlisttmp = glob.glob(datafolder+"/*.csv")
    for f in csvlisttmp:
        csvlist.append(f)
print(csvlist)

['/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_03', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_04', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_02', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_11_08', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_17', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_10', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_01_30  - Sprinkles drive', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_18', '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_27', '/Users/sprinkle/work/d

In [306]:
num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(datafolder, num_of_files))

Total number of datafiles in /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_12 is 159.


## Analysis
### 1. CSV file containing all messages
In this section, we will extract a subset of the CSV-formatted CAN Data in order to produce summary information of a specific drive.

In [309]:
dbcfile = '../examples/newToyotacode.dbc'
metadata = []
for i in range(num_of_files):
    try:
        r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
        print(f'Reading {csvlist[i]}')
        duration1 = r0.dataframe['Time'][len(r0.dataframe)-1] - r0.dataframe['Time'][0]
        duration_str = f'  Duration of this drive is {duration1} seconds ({math.trunc(int(duration1*1000)/(1000*60))} minutes {math.trunc(duration1 % 60)} seconds).'
        start_str = f'  Starting date/time of the drive is {time.ctime(r0.dataframe["Time"][0])}'

        # get the speed timeseries information from the data file
        speed_ts = r0.ts_speed()
        # turn the timeseries into a python array for integration
        # transform from km/hr by multiplying 1000m/1km and 1 hr/3600s to get m/s
        speed_ar = np.array(speed_ts['Message'])*1000/3600
    #     speed_ar[0:-1]*1000/3600
        # find the difference of the time values
    #     dt = np.diff(np.array(speed_ts['Time']))
        dt = np.diff(speed_ts['Time'])
        # trapezoidal integration, divide by 1000 to get total km (rather than m)
        # Commented out: this produces negative values that don't make sense
    #     km_ts = np.trapz(y=speed_ar[0:-1],x=dt)/1000
        km_ts = np.trapz(y=speed_ar,x=np.array(speed_ts['Time']))/1000
        # commented out: this produces incorrect values that seem to be off by a factor of around 2, depending
        km_dx = np.trapz(y=speed_ar,dx=0.02)/1000
        # need to convert km/hr to km/s to get km later when integrating
        distance_str = f'  The trip was {km_ts} km ({km_ts*3.1/5} miles)'
    #     distance_str_dx f'  The dx version is {km_dx} km ({km_dx*3.1/5} miles)'

        #speed_ts['Time']
        #f'Total miles driven is {km_ts}'
    #     print(duration_str)
    #     print(start_str)
    #     print(distance_str)
    #     print('')
        drive = { 'filename': csvlist[i], 'distance_km': km_ts, 'distance_miles': km_ts*3.1/5, 'duration_s': duration1, 
                 'date': time.ctime(r0.dataframe["Time"][0]) }
        metadata.append( drive )
    except:
        print('Unable to process file...')
# print(metadata)

Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439_CAN_Message_Rav4.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1_CAN_Message_Rav4.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-49-54-310111_cb40aa45-de04-4313-bb9b-1bcf10ebaf70_CAN_Message_Rav4.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-12-46-30-126677_8f806f3c-159e-4096-8552-b0aa7a0a681d_CAN_Message_Rav4.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-42-883600_47016aab-73b8-470f-8c10-3e14aa85b582_CAN_Message_Rav4.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-39-54-897

/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe_2.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe_3.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-00-07-789629__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_2.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data_Giraffe.csv


/Users/sprinkle/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/CAN_Data.csv
Unable to process file...
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-00-52-647461__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-02-55-225012__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/0000_00_00/2018-01-28-09-07-22-781083__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_13/2020-02-13-15-37-19-262923__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_13/2020-02-13-08-29-09-380270__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_02_12/2020-02-12-12-42-48-573274__CAN_Message.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadan

In [310]:
# how did we do?
dist=0
for d in metadata:
    dist = dist + d['distance_miles']
print(dist)

560.9846792727091


In [311]:
metadata

[{'filename': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-51-36-833648_a4f27f0c-36b0-4abe-beed-3d548605c439_CAN_Message_Rav4.csv',
  'distance_km': 0.0,
  'distance_miles': 0.0,
  'duration_s': 1.810054063796997,
  'date': 'Fri Oct 25 13:51:36 2019'},
 {'filename': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-57-22-031456_b118f877-5385-4205-894a-a37515307ee1_CAN_Message_Rav4.csv',
  'distance_km': 0.0,
  'distance_miles': 0.0,
  'duration_s': 0.9986419677734375,
  'date': 'Fri Oct 25 13:57:22 2019'},
 {'filename': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2019_10_25/2019-10-25-13-49-54-310111_cb40aa45-de04-4313-bb9b-1bcf10ebaf70_CAN_Message_Rav4.csv',
  'distance_km': 0.0,
  'distance_miles': 0.0,
  'duration_s': 0.9817490577697754,
  'date': 'Fri Oct 25 13:49:54 2019'},
 {'filename': '/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData